In [1]:
import os
print(os.getcwd())

/Users/kevintsai/Documents/GitHub/stanCode/SC201Assignment3


In [2]:
import math
import pandas as pd
TRAIN_FILE = 'titanic_data/train.csv'
TEST_FILE = 'titanic_data/test.csv'

In [115]:
mode = 'Test'
d = pd.read_csv(TRAIN_FILE)
t = pd.read_csv(TEST_FILE)

In [206]:

def data_preprocess(filename: str, data: dict, mode='Train', training_data=None):
	"""
	:param filename: str, the filename to be processed
	:param data: an empty Python dictionary
	:param mode: str, indicating if it is training mode or testing mode
	:param training_data: dict[str: list], key is the column name, value is its data
						  (You will only use this when mode == 'Test')
	:return data: dict[str: list], key is the column name, value is its data
	"""

	d = pd.read_csv(filename)
	if mode == 'Train':	
		feature_name = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
		d = d[feature_name]
		d.dropna(subset=['Age', 'Embarked'],inplace=True)

	elif mode == 'Test':
		feature_name = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
		d = d[feature_name]

		train_age_avg = round(sum(training_data.get('Age')) / len(training_data.get('Age')),3)
		train_fare_avg = round(sum(training_data.get('Fare')) / len(training_data.get('Fare')),3)
		
		d['Age'].fillna(train_age_avg, inplace = True)	
		d['Fare'].fillna(train_fare_avg, inplace = True)	

	# Changing 'male' to 1, 'female' to 0
	d.loc[d.Sex == 'male', 'Sex'] = 1
	d.loc[d.Sex == 'female', 'Sex'] = 0

	# Changing 'S' to 0, 'C' to 1, 'Q' to 2
	d.loc[d.Embarked == 'S', 'Embarked'] = 0
	d.loc[d.Embarked == 'C', 'Embarked'] = 1
	d.loc[d.Embarked == 'Q', 'Embarked'] = 2


	data = d.to_dict(orient='list')

	return data


def one_hot_encoding(data: dict, feature: str):
	"""
	:param data: dict[str, list], key is the column name, value is its data
	:param feature: str, the column name of interest
	:return data: dict[str, list], remove the feature column and add its one-hot encoding features
	"""
	############################
	#                          #
	#          TODO:           #
	#                          #
	############################
	
	"""
	unique_list = []
	
	for x in data.get(feature):
		if x not in unique_list:
			unique_list.append(x)
	
	print(unique_list)

	data = pd.DataFrame(data)

	for i in unique_list:
		data[feature + '_' + str(i)] = 0	
		data.loc[data[feature] == i, feature + '_' + str(i)] = 1
	
	data.pop(feature)
	data = data.to_dict(orient='list')
	
	"""
	data = pd.DataFrame(data)
	if feature == 'Sex':
		data['Sex_0'] = 0
		data['Sex_1'] = 0

		data.loc[data.Sex == 1, 'Sex_1'] = 1
		data.loc[data.Sex == 0, 'Sex_0'] = 1
		data.pop('Sex')
	
	elif feature == 'Pclass':
		data['Pclass_0'] = 0
		data['Pclass_1'] = 0
		data['Pclass_2'] = 0

		data.loc[data.Pclass == 1, 'Pclass_0'] = 1
		data.loc[data.Pclass == 2, 'Pclass_1'] = 1
		data.loc[data.Pclass == 3, 'Pclass_2'] = 1
		
		data.pop('Pclass')
	
	elif  feature == 'Embarked':
		data['Embarked_0'] = 0
		data['Embarked_1'] = 0
		data['Embarked_2'] = 0

		data.loc[data.Embarked == 0, 'Embarked_0'] = 1
		data.loc[data.Embarked == 1, 'Embarked_1'] = 1
		data.loc[data.Embarked == 2, 'Embarked_2'] = 1
		
		data.pop('Embarked')
	
	data = data.to_dict(orient='list')
	
	return data


def normalize(data: dict):
	"""
	:param data: dict[str, list], key is the column name, value is its data
	:return data: dict[str, list], key is the column name, value is its normalized data
	"""
	############################
	#                          #
	#          TODO:           #
	#                          #
	############################
	data = pd.DataFrame(data)

	data = (data - data.min()) / (data.max() - data.min())

	data = data.to_dict(orient='list')

	return data


def increment(d1, scale, d2):
    """
    Implements d1 += scale * d2 for sparse vectors.
    @param dict d1: the feature vector which is mutated.
    @param scale: float, scale value of d2 to add onto the corresponding value of d1
    @param dict d2: a feature vector.
    """
    # BEGIN_YOUR_CODE (our solution is 2 lines of code, but don't worry if you deviate from this)
    for key, value in d2.items():
        d1[key] = d1.get(key, 0) + scale * value
    # END_YOUR_CODE


############################################################


def dotProduct(d1, d2):
    """
    @param dict d1: a feature vector. Key is a feature (string); value is its weight (float).
    @param dict d2: a feature vector. Key is a feature (string); value is its weight (float)
    @return float: the dot product between d1 and d2
    """
    if len(d1) < len(d2):
        return dotProduct(d2, d1)
    else:
        # BEGIN_YOUR_CODE (our solution is 1 line of code, but don't worry if you deviate from this)
        return sum(d1.get(key, 0)*value for key, value in d2.items())
        # END_YOUR_CODE



In [186]:
#def test5_0():
ans = 0.8104
train_data = data_preprocess('titanic_data/train.csv', {})
train_data = one_hot_encoding(train_data, 'Sex')
train_data = one_hot_encoding(train_data, 'Pclass')
train_data = one_hot_encoding(train_data, 'Embarked')
labels = train_data.pop('Survived')
labels = list(int(labels[i]) for i in range(len(labels)))
train_data = normalize(train_data)
    #weights = titanic_level1.learnPredictor(train_data, labels, 1, 100, 0.1)
    #test = round(evaluatePredictor(train_data, labels, weights, 2), 4)
    #grader.require_is_equal(ans, test)
#grader.add_basic_part('test5_0', test5_0, max_points=10, max_seconds=10, description="Milestone 5 - Classification Model degree 1")


In [204]:
len(train_data['Age'])

712

In [207]:

def learnPredictor(inputs: dict, labels: list, degree: int, num_epochs: int, alpha: float):
	"""
	:param inputs: dict[str, list], key is the column name, value is its data
	:param labels: list[int], indicating the true label for each data
	:param degree: int, degree of polynomial features
	:param num_epochs: int, the number of epochs for training
	:param alpha: float, known as step size or learning rate
	:return weights: dict[str, float], feature name and its weight
	"""
	# Step 1 : Initialize weights
	weights = {}  # feature => weight
	keys = list(inputs.keys())
	if degree == 1:
		for i in range(len(keys)):
			weights[keys[i]] = 0
	elif degree == 2:
		for i in range(len(keys)):
			weights[keys[i]] = 0
		for i in range(len(keys)):
			for j in range(i, len(keys)):
				weights[keys[i] + keys[j]] = 0


	def sigmoid(k):
		return 1 / (1 + math.exp(-k))
	# Step 2 : Start training
	# TODO:
	# Step 3 : Feature Extract
	# TODO:
	# Step 4 : Update weights
	# TODO:

	keys = list(inputs.keys())
	length = len(inputs[keys[0]])

	for epoch in range(num_epochs):
		for i in range(length):
			x = {k:v[i] for k, v in inputs.items()}
			h = sigmoid(dotProduct(weights, x))
			scale = -alpha*(h-labels[i])
			increment(weights, scale, x)

	return weights

In [208]:
weights = learnPredictor(train_data, labels, 1, 100, 0.1)

TypeError: unsupported operand type(s) for -: 'float' and 'list'

In [195]:
weights

{'Age': 0,
 'SibSp': 0,
 'Parch': 0,
 'Fare': 0,
 'Sex_0': 0,
 'Sex_1': 0,
 'Pclass_0': 0,
 'Pclass_1': 0,
 'Pclass_2': 0,
 'Embarked_0': 0,
 'Embarked_1': 0,
 'Embarked_2': 0}